<a href="https://colab.research.google.com/github/Odera2023/RAG/blob/main/odera_rag_practical1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Install & Import Dependencies

In [1]:
!pip install langchain langchain-core langchain-text-splitters chromadb langchain_nvidia_ai_endpoints

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 17.1 MB/s eta 0

In [2]:
# Imports
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
import chromadb
from langchain.vectorstores import Chroma

ModuleNotFoundError: No module named 'langchain.text_splitter'

### 2. Initialize Embeddings + LLM

In [ ]:
# Embeddings
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# Instruction LLM (chat model)
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

### 3. Load a Document (Simple Text)

In [ ]:
!pip install langchain langchain-core langchain-text-splitters chromadb langchain_nvidia_ai_endpoints pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load a PDF file
loader = PyPDFLoader("LangChain.pdf")
documents = loader.load()

documents[:1]   # Preview the first page/document chunk


[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-11-10T11:43:58+00:00', 'author': '', 'keywords': '', 'moddate': '2024-11-10T11:43:58+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'LangChain.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='LangChain\nVasilios Mavroudis\nAlan Turing Institute\nvmavroudis@turing.ac.uk\nAbstract. LangChain is a rapidly emerging framework that offers a ver-\nsatile and modular approach to developing applications powered by large\nlanguage models (LLMs). By leveraging LangChain, developers can sim-\nplify complex stages of the application lifecycle—such as development,\nproductionization, and deployment—making it easier to build scalable,\nstateful, and contextually aware applications. It provides tools for han-\ndling chat models, integrating retrieval

### 4. Chunk the Document

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
len(chunks)

84

### 5. Create & Persist Chroma Vector Store

In [ ]:
chroma_client = chromadb.Client()

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedder,
    client=chroma_client,
    collection_name="rag_tutorial"
)

### 6. Create Retriever

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

### 7. Build the RAG Runnable Pipeline

In [ ]:
# Prompts and output parsers
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnable
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

In [ ]:
# Define chat prompt template
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

# Build a RAG chain using LCEL (LangChain Expression Language)
# LCEL allows chaining runnables and parsing outputs
rag_chain = RunnableParallel({
    "context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
    "question": RunnablePassthrough()
}) | (
    lambda inputs: context_prompt.format(context=inputs["context"], question=inputs["question"])
) | instruct_llm | StrOutputParser()


In [ ]:
# Manual invoke to test the RAG chain
response = rag_chain.invoke("What does the document say about LangChain RAG?")
print("Response:\n", response)

Response:
 LangChain RAG, or Retrieval-Augmented Generation, is a feature that enables language models to use up-to-date external knowledge bases to enhance the accuracy of their responses. LangChain's implementation of RAG uses document loaders and text splitters to preprocess documents for efficient retrieval, embedding models and vector stores to embed documents into vector spaces for similarity-based retrieval, and supports structured outputs like JSON for easy integration with other applications. Furthermore, it maintains conversation memory for continuity, especially useful for applications requiring persistent context such as customer support.


In [ ]:
# Ask user for a question using standard Python input
user_question = input("Enter your question about the document: ")

# Run the RAG chain with user input
user_response = rag_chain.invoke(user_question)
print("Assistant Response:\n", user_response)

Enter your question about the document:  What is Langserve from the document


Assistant Response:
 LangServe is a component of the LangChain ecosystem that focuses on deploying large language model (LLM) applications as scalable REST APIs. It allows developers to create production-grade APIs that can be interacted with by external systems and users. One of the main strengths of LangServe is its ability to handle multiple API requests simultaneously, providing consistent performance even under high traffic volumes. This makes it ideal for production environments where maintaining low response times is crucial. Furthermore, LangServe integrates with LangSmith for observability features like tracing, logging, and performance monitoring, enabling developers to track API performance and optimize applications based on real-time data.


### Conversation State (Chat Memory)

In [ ]:
# Simple conversation history
chat_history = []

def conversational_rag(user_query):
    """Runs RAG with conversation history (very simple state)."""
    global chat_history

    # Join past messages into a readable history string
    history_text = "\n".join([
        f"{h['role']}: {h['content']}" for h in chat_history
    ])

    # Append conversation history to user query
    final_query = f"""
Conversation so far:
{history_text}

User: {user_query}
""".strip()

    # Run RAG chain
    response = rag_chain.invoke(final_query)

    # Save turn to history
    chat_history.append({"role": "user", "content": user_query})
    chat_history.append({"role": "assistant", "content": response})

    return response


#### Using Langchain Messages

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


In [ ]:
chat_history = [
    SystemMessage(content="You are a helpful assistant.")
]

In [ ]:
def conversational_rag(user_query):
    """RAG with proper message objects instead of raw dicts."""

    global chat_history

    # Convert message objects to readable text for the RAG chain
    history_text = "\n".join([
        f"{type(msg).__name__}: {msg.content}"
        for msg in chat_history
    ])

    # Build final combined query
    final_query = f"""
Conversation so far:
{history_text}

User: {user_query}
""".strip()

    # Run RAG chain
    response = rag_chain.invoke(final_query)

    # Update history using proper LangChain message classes
    chat_history.append(HumanMessage(content=user_query))
    chat_history.append(AIMessage(content=response))

    return response

In [ ]:
print("🔵 RAG Chatbot is ready! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("🟢 Chatbot: Goodbye! 👋")
        break

    bot_response = conversational_rag(user_input)

    print("\n🟢 Chatbot:", bot_response, "\n")

🔵 RAG Chatbot is ready! Type 'exit' to quit.



You:  Hi there, what is the name of your model. I am Ebuka



🟢 Chatbot: Hello Ebuka! I'm an assistant built using a model from LangChain. It's a system designed for complex, multi-turn conversations like ours, helping me maintain context and provide contextually aware responses. It's a pleasure to meet you! 



You:  what is my name again?



🟢 Chatbot: Hello there! Your name is Ebuka, remember? I'm here to assist you today. Let me know if you have any questions or need help with something! 



You:  what does the document say about LangServe, and reference my name in your response



🟢 Chatbot: LangServe is a component of LangChain that's designed for scalable API deployment. Essentially, it enables the integration of complex language models into various applications, Ebuka. This makes it easier for developers to create functional and contextually aware solutions. 



You:  quit


🟢 Chatbot: Goodbye! 👋


## LLM-as-Judge

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### 1. Judge Prompt Template

In [ ]:
judge_prompt = ChatPromptTemplate.from_template(
    """
You are an evaluation assistant.

You will judge whether the RAG answer is correct based ONLY on the provided context.

Context:
{context}

User Question:
{question}

RAG Answer:
{answer}

Evaluate the correctness on a scale of 1 to 5.
Explain your reasoning in 2–3 sentences.

Return your answer in this format:
Score: <number>
Reason: <your explanation>
"""
)

### 2. Create the Judge Chain (LCEL)

In [ ]:
judge_chain = judge_prompt | instruct_llm | StrOutputParser()


### 3. Helper Function to Retrieve Context + Run Judge

In [ ]:
def evaluate_answer(question, rag_answer):
    """
    Runs the LLM-as-a-Judge using the retrieved context and RAG answer.
    """
    # Retrieve context for the question
    retrieved_docs = retriever.invoke(question)
    context_text = "\n\n".join([d.page_content for d in retrieved_docs])

    # Run judge chain
    judge_result = judge_chain.invoke({
        "context": context_text,
        "question": question,
        "answer": rag_answer
    })

    return judge_result

In [ ]:
###